# Linear Regression

- Linear Data
- Linear Model
- y = 2x+3

## 1. Import Required Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable

# **visdom usage**

In [2]:
# 코렙에서 visdom 사용 방법

# !npm install -g npm to update!
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

import time
time.sleep(5)
! cat url.txt

import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.833s
your url is: https://splendid-cow-83.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.76ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.44ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.77ms


## 2. Generate Data

In [7]:
num_data = 1000
num_epoch = 1000

noise = init.normal(torch.FloatTensor(num_data, 1), std=1)
x = init.uniform(torch.Tensor(num_data, 1), -10, 10)

y = 2 * x + 3
y_noise = 2 * x + 3 + noise

x.size(), y.size()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """


(torch.Size([1000, 1]), torch.Size([1000, 1]))

In [8]:
# visualize data with visdom

input_data = torch.cat([x, y_noise], 1)

win=vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-10,
        xtickmax=10,
        xtickstep=1,
        ytickmin=-20,
        ytickmax=20,
        ytickstep=1,
        markersymbol='dot',
        markersize=5,
        markercolor=np.random.randint(0, 255, num_data),
    ),
)

# Tensor(n,1) != Tensor(n, 1) 버그 있음
vis.line(
    X=x.reshape(x.shape[0]), 
    Y=y.reshape(y.shape[0]), 
    win=win)

'window_3a50d0eb744bc8'

### 3. Model & Optimizer

In [9]:
# 단순 선형 회귀 생성 및 초기화
# input_dim=1, output_dim=1
model = nn.Linear(1, 1)
output = model(Variable(x))

loss_func = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# lr (Learning Rate) : 미분값을 얼마만클 이동시킬 것인지를 결정
# 최적화 알고리즘 : lossㄹ를 줄이는 알고리즘
# SGD : loss 의 미분을 이용해서 loss 의 최소값을 찾는 알고리즘에 해당됨

### 4. Train

In [17]:
# train
loss_arr = []
label = Variable(y_noise)
for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x))

    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

    loss_arr.append(loss.data.numpy())

tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.0522, grad_fn=<MseLossBackward0>)
tensor(1.05

### 5. Check Trained Parameters

In [18]:
param_list = list(model.parameters())
print(param_list[0].data, param_list[1].data)

tensor([[1.9988]]) tensor([2.9654])


### 6. Visualize output

In [22]:
win_2=vis.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-10,
        xtickmax=10,
        xtickstep=1,
        ytickmin=-20,
        ytickmax=20,
        ytickstep=1,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=5,
    ),
)

vis.line(
    X=x.reshape(x.shape[0]),     
    Y = output.data.reshape(output.data.shape[0]),
    win = win_2,
    opts=dict(
        xtickmin=-15,
        xtickmax=10,
        xtickstep=1,
        ytickmin=-300,
        ytickmax=200,
        ytickstep=1,
        markersymbol='dot',
    ),
)

'window_3a50d522f26376'

### 7. Visualize Loss Graph

In [25]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win2=vis.line(
    X = x.reshape(x.shape[0]),
    Y = loss_data.reshape(loss_data.shape[0]),
)